In [1]:
# Load the memory profiler
%load_ext memory_profiler
import numpy as np
from inverseDesign import kgrid
from defineCrystal import L3Crystal 
from legume.primitives import fmap
import legume
import autograd.numpy as npa
from autograd import grad, value_and_grad
from inverseDesign import placeParams
legume.set_backend('autograd')
import time

In [2]:
def gme_simple(dx,gmax, options, kpoints, f_lb=0):
    phc = L3Crystal(dx=dx,Nx=Nx,Ny=Ny,dslab=.5,n_slab=12.04,ra=.25)
    options['compute_im'] = False
    gme = legume.GuidedModeExp(phc, gmax=gmax)
    gme.run(kpoints=kpoints, **options)
    indmode = npa.nonzero(gme.freqs[0, :] > f_lb)[0][0]
    fims = []
    for ik in range(kpoints[0, :].size):
        (freq_im, _, _) = gme.compute_rad(ik, [indmode])
        fims.append(freq_im)
    return (gme, npa.array(fims), indmode)

In [3]:
options = {'verbose': False, 'gradients': 'approx',
           'numeig': 5,       # get 5 eigenvalues
           'eig_sigma': 0.26,  # closest to f = 0.28
           'eig_solver': 'eigsh',
           'compute_im': False
          }

pstart = [0.0,0.0]
dx = {(0,2):0,(0,-2):0}
gmax = 1
Nx = 20; Ny = 10
ks = kgrid(Nx=Nx,Ny=Ny)

In [4]:
def fim_kavg(params):
    dx,dy,dr = placeParams(params,dx={(0,2):0,(0,-2):0})
    (gme, fims, _) = gme_simple(dx, gmax, options, ks)
    # Scale for easier readability
    return npa.mean(fims)*1e6

def of_kavg_fmap(params):
    # A function factory to make a list of functions for every k-point
    def fim_factory(ik):
        def fim(params):
            dx,dy,dr = placeParams(params,dx={(0,2):0,(0,-2):0})
            (gme, freq_im, _) = gme_simple(dx,gmax,options,np.array([[ks[0,ik]],[ks[1,ik]]]))
            return freq_im
        return fim

    fims = fmap([fim_factory(ik=ik) for ik in range(ks[0].size)],params)
    return npa.mean(fims)*1e6

In [7]:
obj_grad = value_and_grad(fim_kavg)

t = time.time()
%memit (fim, grad_a) = obj_grad(pstart)
print(time.time() - t,fim,grad_a)

peak memory: 1016.12 MiB, increment: 849.52 MiB
3.262972831726074 0.13196128236048973 [array(-6.80898115e-05), array(6.80898115e-05)]


In [5]:
obj_grad = value_and_grad(of_kavg_fmap)

t = time.time()
%memit (fim, grad_a) = obj_grad(pstart)
print(time.time() - t,fim,grad_a)

peak memory: 690.52 MiB, increment: 525.22 MiB
6.419666051864624 0.13196128236048973 [-6.61839132e-05  6.61839132e-05]
